In [1]:
# Classification, Intrinsic dimension, Degeneracy
%run SetUp.ipynb

## Configuration 
Title = "Rebuttal" # general two layer
device = "cuda:1"
dataset_type = "Training set" # "Training + Test set"
data_balace = 1
selected_class = 1 ################ selected clas ##### 

pt_option=False
dataset = "MNIST"
random_ratio = 0/100   ## 001 / 010 / 025 / 050 / 075 / 100
width = 10
width_increment = 30


###############################
num_classes = 2 # binary classification
Goal_Accuracy = 99.99
first_positive_init = False ## if False, this approximate the class : {selected_class}
############################### if Ture, approximate the complement : {selected_class}^c


if dataset == "MNIST":
    d = 28*28
    n = 60000
    data_balance = 1 ## the lambda value
    Epochs = 100*1000
    start_pruning = 9999 # pretraining

elif dataset == "FashionMNIST":
    d = 28*28
    n = 60000
    data_balance = 1 ## the lambda value
    Epochs = 100*1000
    start_pruning = 9999 # pretraining

elif dataset == "CIFAR10":
    d = 3*32*32
    n = 50000
    data_balance = 1 ## the lambda value
    Epochs = 100*1000
    start_pruning = 9999 # pretraining
    
else:
    raise ValueError("Wrong Dataset !")
# n = 1000 # for debug, if need



# create directory
output_path = f"./Results/{Title}"
create_directory(output_path)
# construct the dataset
trn_X, trn_Y, test_X, test_Y = load_dataset(dataset, n)


if dataset_type == "Training + Test set" : 
    ########### trn + test set #############
    trn_X = torch.cat((trn_X, test_X)) ##### combine test set and trn set !
    trn_Y = torch.cat((trn_Y, test_Y)) ##### 
    ########################################
elif dataset_type == "Training set":
    pass
else:
    raise ValueError(f"Wrong type for the dataset_type : {dataset_type}")


trn_Y = (trn_Y == selected_class).float()# change to binary class ## label 1 for the selected class.
test_Y = (test_Y == selected_class).float()
n = len(trn_X)


###################################
####### random label ##############
print(f"\n\n\nThis is experiment with random label, random_ratio: {random_ratio}, Goal accuracy: {Goal_Accuracy}")
trn_Y_clone = trn_Y.clone()
_k = 0
while True:
    trn_Y_clone = trn_Y.clone()
    random_Y = ( torch.rand(len(trn_Y)).to(device) < (trn_Y.sum() / len(trn_Y)) ).float().to(device) ######################### random dataset !!!!
    random_index = (torch.rand(len(trn_Y)) < random_ratio)
    trn_Y_clone[random_index] = random_Y[random_index].clone()
    _k+=1
    if trn_Y_clone.sum() == len(trn_Y[trn_Y>0]) :
        class_overlapped = (trn_Y[trn_Y==1] == trn_Y_clone[trn_Y==1]).sum().item()
        print("Class Overlapped :", class_overlapped, "/", len(trn_Y[trn_Y==1]))
        trn_Y = trn_Y_clone
#         print(trn_Y_clone.sum())
#         print(_k)
        break
##########################################
###################################




This is experiment with random label, random_ratio: 0.0, Goal accuracy: 99.99
Class Overlapped : 6742 / 6742


Network Architecture

In [2]:
# Network setting # d -> d1 -> 1
class polytope(nn.Module):
    def __init__(self, width, output_class=1, positive_init=False, small_norm_init=False):
        super(polytope, self).__init__()
        self.fc0 = nn.Linear(d, width)
        self.fc1 = nn.Linear(width, output_class, bias=False)
        self.width = width # width
        self.epoch = 0
        self.data_balance = data_balance
        self.bias = (1 - 2*positive_init)*5 # +-5
        self.positive_init = positive_init
        self.output_class = output_class
        self.small_norm_init = small_norm_init
        self.device = device
        
        # initialization
        if small_norm_init:
            self.change_layer_weights(layer=0, W=1.5/self.width*torch.randn_like(self.W(0)), b=0.6/self.width*(self.b(0))) # multiply 0.01 
            
        if positive_init:
            # if positivie init,  all v_k are positive.
            self.fc1.weight = nn.Parameter(torch.sqrt((self.W(0).norm(dim=1)**2 + self.b(0)**2).view(1,-1)+1))
        else:
            # else, to all v_k are negative.
            self.fc1.weight = nn.Parameter(-torch.sqrt((self.W(0).norm(dim=1)**2 + self.b(0)**2).view(1,-1)+1))
        
        # declare the optimizer
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        
    def forward(self, x):
        self.g1 = self.fc0(x)
        self.h1 = F.relu(self.g1)
        self.g2 = self.fc1(self.h1) + self.bias
        return self.g2.squeeze()
    
    def W(self, layer):
        if layer ==0:
            output = self.fc0.weight
        elif layer ==1:
            output = self.fc1.weight
        return output.clone()
    def b(self, layer):
        if layer ==0:
            output = self.fc0.bias
        elif layer ==1:
            output = self.fc1.bias
        return output if output == None else output.clone()
    
    def activation_pattern(self, x):
        self.forward(x)
        return (self.h1>0).float()
    
    def change_layer_weights(self, layer, W, b):
        if self.b(layer) == None : # no bias term
            if W.shape == self.W(layer).shape and self.b(layer) == b:
                if layer ==0:
                    self.fc0.weight = nn.Parameter(W.to(self.device))
                elif layer ==1:
                    self.fc1.weight = nn.Parameter(W.to(self.device))
            else:
                raise ValueError("wrong shape of input tensors")
        else:
            if W.shape == self.W(layer).shape and b.shape == self.b(layer).shape:
                if layer ==0:
                    self.fc0.weight = nn.Parameter(W.to(self.device))
                    self.fc0.bias = nn.Parameter(b.to(self.device))
                elif layer ==1:
                    self.fc1.weight = nn.Parameter(W.to(self.device))
    #                 self.fc1.bias = nn.Parameter(b)
            else:
                raise ValueError("wrong shape of input tensors")
    
    
    # Training
    def train(self, repetition, trn_X_index):
        self.fail = False
        self.trn_loss_tr = np.empty(0)
        self.test_loss_tr = np.empty(0)
        self.trn_acc_tr = np.empty(0)
        self.test_acc_tr = np.empty(0)

        class0_outPoly_min = 1
        test_acc = 0
        test_loss = torch.zeros(1)
        print(f"=================================   Start Training. Repetition: {repetition+1}   ==================================")
        time.sleep(1)

        log_period = 500
        for epoch in tqdm(range(Epochs)) :
            self.epoch = epoch
            
            # since class may be imbalance
            loss =  criterion(net(trn_X[trn_X_index][(trn_Y[trn_X_index]==float(positive_init)).view(-1)]).squeeze(),
                              trn_Y[trn_X_index][(trn_Y[trn_X_index]==float(positive_init)).view(-1)]) / len((trn_Y[trn_X_index]==float(positive_init)).view(-1))
            loss += criterion(net(trn_X[trn_X_index][(trn_Y[trn_X_index]==float(not positive_init)).view(-1)]).squeeze(), 
                              trn_Y[trn_X_index][(trn_Y[trn_X_index]==float(not positive_init)).view(-1)]) / len((trn_Y[trn_X_index]==float(not positive_init)).view(-1)) * self.data_balance
            loss *= len((trn_Y[trn_X_index]).view(-1))
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
#             self.pruning(trn_X[trn_X_index])


            if epoch ==0 or epoch % log_period == (log_period-1) :
                self.trn_loss_tr = np.append(self.trn_loss_tr, loss.item())
                self.test_loss_tr = np.append(self.test_loss_tr, test_loss.item())

                trn_acc = ((net(trn_X[trn_X_index])>0).float() == trn_Y[trn_X_index]).sum().item() / len(trn_X[trn_X_index]) *100
                self.trn_acc_tr = np.append(self.trn_acc_tr, trn_acc)
                self.test_acc_tr = np.append(self.test_acc_tr, test_acc)
                class0_inPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]!=float(self.positive_init)).view(-1)]) == self.bias).sum().item()
                class1_inPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]==float(self.positive_init)).view(-1)]) == self.bias).sum().item()
                class0_outPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]!=float(self.positive_init)).view(-1)]) != self.bias).sum().item()
                class1_outPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]==float(self.positive_init)).view(-1)]) != self.bias).sum().item()
                writer.add_scalars("ReLU/Loss", {
                    "Trn_Loss": loss.item(),
                    "Test_Loss": test_loss.item(), }
                                   , epoch+1)
                writer.add_scalars("ReLU/Accuracy", {
                    "Trn_acc": trn_acc,
                    "Test_acc": test_acc, }
                                   , epoch+1)
                writer.add_scalars("ReLU/InOutClass", {
                    "class0_inPoly": class0_inPoly,
                    "class0_outPoly": class0_outPoly,
                    "class0_inPoly": class1_inPoly,
                    "class0_outPoly": class1_outPoly,}
                                   , epoch+1)
                if pt_option :
                    print(f"Epoch: {epoch+1 :>5} | width: {self.width} | loss: {loss.item() :.4f} | ACC: {trn_acc:.3f}", end="")
                    print(f" | Wmin: {self.fc1.weight.abs().min():.1f} | class0_inPoly : {class0_inPoly}", end="")
                    print(f" | class0_outPoly : {class0_outPoly} | class1_inPoly : {class1_inPoly} | class1_outPoly : {class1_outPoly}")
                
                if class0_outPoly == 0: # a good network. save.
                    class0_outPoly_min = 0
                    torch.save(self.state_dict(), folder_name+f"/Rep{repetition+1}_saved_net_width_{width}_min.pt") # save a good polytope
                torch.save(self.state_dict(), folder_name+f"/Rep{repetition+1}_saved_net_width_{width}.pt") # save a good polytope
                
                if self.fc1.weight.sign().sum().abs() !=  self.width: # there is a flipped sign.
                    print("!!! Sign flipped !!") 
                    self.fail = True # skip, train a new network for this round
                    break
                
                if self.width == 0 : # there is no active neuron.
                    print("!!! All neruons have been removed !!") 
                    self.fail = True # skip, train a new network for this round
                    break
                
                if (class1_inPoly > (100 - Goal_Accuracy)/100 * len(trn_Y)) and (epoch > 5000) :
                    print(f"Fail, more than {(100 - Goal_Accuracy) :.2f}% wrong class in the polytope with width {self.width}") 
                    self.fail = True
                    break
                    

                
                if (n - class0_outPoly - class1_inPoly) / n * 100 >= Goal_Accuracy :
                    print(f"Found a {Goal_Accuracy}% convex polytope cover with width {self.width}, escape the training loop") 
                    self.fail = False
                    break

                # Pruning and merging
                self.pruning(trn_X[trn_X_index]) if epoch > start_pruning else None ## pruning after pre-training
        
        if class0_outPoly_min ==0:
            self.load_state_dict(torch.load(folder_name+f"/Rep{repetition+1}_saved_net_width_{width}_min.pt")) # save a good polytope
        
        # if it fails to achieve convergence : 
        if (class0_outPoly > (100 - Goal_Accuracy)/100 * len(trn_Y) ):
            print(f"!!! It is failed to find a perfect polytope with width {self.width}!!!!") 
            self.fail = True # skip, train a new network for this round
#         else: # load the saved network with a good polytope
#             self.load_state_dict(torch.load(folder_name+f"/Rep{repetition+1}_saved_net_width_{width}.pt")) # save a good polytope
# #             self.eval()
                                 
        self.save_loss_graph()
        self.save_result_txt()
        time.sleep(1)
        print(f"===================================   Training finished, Repetition: {repetition+1}  ================================== \n\n")
        
    
    def save_loss_graph(self):
        # plot and save the graphs
        # PLOT THE LOSS GRAPH
        plt.plot(self.trn_loss_tr, label="Train")
        plt.plot(self.test_loss_tr, label="Test")
        plt.xlabel("Iterations (k)")
        plt.ylabel("Loss")
        # plt.yscale("log")
        plt.legend()
        plt.title(f"Rep{repetition+1}_Loss")
        plt.savefig(folder_name+"/loss_fig_fullview.png")
#         plt.show() if pt_option else None
        plt.close()

        # PLOT THE ACCURACY GRAPH
        plt.plot(self.trn_acc_tr, label="Train")
        plt.plot(self.test_acc_tr, label="Test")
        plt.xlabel("Iterations (k)")
        plt.ylabel("Accuracy")
        plt.legend()
        plt.title(f"Rep{repetition+1}_Accuracy")
        plt.savefig(folder_name+"/Accuracy.png")
#         plt.show() if pt_option else None
        plt.close()
    
    # Save result txt file.
    def save_result_txt(self):
        class0_inPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]!=float(self.positive_init)).view(-1)]) == self.bias).sum().item()
        class1_inPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]==float(self.positive_init)).view(-1)]) == self.bias).sum().item()
        class0_outPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]!=float(self.positive_init)).view(-1)]) != self.bias).sum().item()
        class1_outPoly = (self.forward(trn_X[trn_X_index][(trn_Y[trn_X_index]==float(self.positive_init)).view(-1)]) != self.bias).sum().item()
        
        f = open(f"./{folder_name}/Rep{repetition+1}_result.txt", 'w')
        f.write(
        f"""
            This is the CFG file.

            # Accuracy
            Trn_acc : {self.trn_acc_tr[-1] :.3f}
            Test_acc : {self.test_acc_tr[-1] :.3f}

            # Loss
            Trn_loss : {self.trn_loss_tr[-1].item() :.4f} 
            Test_loss : {self.test_loss_tr[-1].item() :.4f} 


            # dataset
            dataset = {dataset}
            n = {n} # number of total data
            d = {d} # dimension
            num_class = {num_classes} # the class, used in this binary classification
            starting_width = {width}
            final_width = {self.width}
            # Network
            {net}
            
            # Polytope
            number of data used in this training (Class 0 / 1): {class0_inPoly+class0_outPoly} / {class1_inPoly+class1_outPoly}
            Class 0 in Polytope : {class0_inPoly}
            Class 0 out Polytope : {class0_outPoly}
            Class 1 in Polytope : {class1_inPoly}
            Class 1 out Polytope : {class1_outPoly}
            
            # optimization
            Epochs = {Epochs}
            lr = {lr}
            optimizer = {self.optimizer}
            Last epoch = {self.epoch+1}
            
            ## trn_X_index
            {trn_X_index}
        """
        )
        f.close()
    
    def pruning(self, dataset):
        width_before = self.width
        dataset_activation_pattern = self.activation_pattern(dataset).sum(dim=0)
        if dataset_activation_pattern.max() == len(dataset) or dataset_activation_pattern.min() == 0 : 
            W0 = self.W(layer=0).detach()
            b0 = self.b(layer=0).detach()
            W1 = self.W(layer=1).detach()
            activation_filter = ((self.activation_pattern(dataset).sum(dim=0) < len(dataset)).float()
                                 * (self.activation_pattern(dataset).sum(dim=0) > 0).float()
                                 * (self.W(layer=1).abs()>0.1).float().view(-1)
                                )
            # change polytope width 
            self.width = activation_filter.sum().long().item()
            width_after = self.width
            self.fc0 = nn.Linear(d, self.width)
            self.fc1 = nn.Linear(self.width, self.output_class, bias=False)
            self.change_layer_weights(layer=0, W=W0[(activation_filter == 1)], b=b0[activation_filter == 1])
            self.change_layer_weights(layer=1, W=W1.view(-1)[activation_filter == 1].view(1,-1), b=None)
            text = "(removing)"
            print(f"\t{text:<15}The network is prunned, width : {width_before} --> {width_after}") if pt_option else None
            
        ## merging vectors with same activation patterns ##
        width_activation_pattern = self.activation_pattern(dataset) # len(dataset) x width
        width_activation_pattern_unique = width_activation_pattern.unique(dim=1).t()
        width_before = self.width
        if len(width_activation_pattern_unique) < self.width :
            self.width = len(width_activation_pattern_unique) 
            width_after = self.width
            W0 = self.W(layer=0).detach()
            b0 = self.b(layer=0).detach()
            W1 = self.W(layer=1).detach()
            self.fc0 = nn.Linear(d, self.width)
            self.fc1 = nn.Linear(self.width, self.output_class, bias=False)
            self.to(device)
            # build weight matrices
            new_W0 = self.W(layer=0).detach()
            new_b0 = self.b(layer=0).detach()
            new_W1 = self.W(layer=1).detach()
            for index, pattern in enumerate(width_activation_pattern_unique):
                new_W0[index] = W0[(width_activation_pattern.t() == pattern).prod(dim=1)==1].sum(dim=0)
                new_b0[index] = b0[(width_activation_pattern.t() == pattern).prod(dim=1)==1].sum(dim=0)
                new_W1[0][index] = W1[0][(width_activation_pattern.t() == pattern).prod(dim=1)==1].sum(dim=0)
            self.change_layer_weights(layer=0, W=new_W0, b=new_b0)
            self.change_layer_weights(layer=1, W=new_W1, b=None)
            text = "(merging)"
            print(f"\t{text:<15}The network is merged, width : {width_before} --> {width_after}") if pt_option else None

        ###### original #######
        smallest_norm_neuron_index = self.width # not index yet.
        smallest_norm = np.inf
        width_activation_pattern = self.activation_pattern(dataset) # len(dataset) x width
        _count = 0
        for neuron_index in range(self.width): # for loop
            activated_index = (width_activation_pattern[:, neuron_index] == 1)
            # index 가 activated 된 data들의 activation pattern ## 이게 2 이상이면 제거해버리면 됨 !
            if self.activation_pattern(dataset[activated_index]).sum(dim=1).min().item() >=2 :
                # this neuron can be removed.
                _count += 1
                this_round_norm = (self.W(layer=0)[neuron_index].norm() * self.W(layer=1).squeeze()[neuron_index]).item() ## v||w||
#                 this_round_norm = self.W(layer=1).squeeze()[neuron_index].item() ## v
                if this_round_norm < smallest_norm:
                    smallest_norm_neuron_index = neuron_index
                    smallest_norm = this_round_norm
        if smallest_norm_neuron_index < self.width:
            # remove the small-norm redundant neuron, remove only one neuron at once
            width_before = self.width
            width_after = self.width - 1
            W0 = self.W(layer=0).detach()
            b0 = self.b(layer=0).detach()
            W1 = self.W(layer=1).detach()
            self.fc0 = nn.Linear(d, width_after)
            self.fc1 = nn.Linear(width_after, self.output_class, bias=False)
            self.to(device)        
            new_W0 = torch.cat((W0[:smallest_norm_neuron_index], W0[smallest_norm_neuron_index+1:]), dim=0)
            new_b0 = torch.cat((b0[:smallest_norm_neuron_index], b0[smallest_norm_neuron_index+1:]), dim=0)
            new_W1 = torch.cat((W1[:,:smallest_norm_neuron_index], W1[:,smallest_norm_neuron_index+1:]), dim=1)
            self.change_layer_weights(layer=0, W=new_W0, b=new_b0)
            self.change_layer_weights(layer=1, W=new_W1, b=None)
            self.width = width_after
            text = "(pruning)"
            print(f"\t{text:<15}There are {_count} redundant neurons, and one has been removed") if pt_option else None
        
        
        ### rescaling function ## here, we don't need to rescale the neuron.
        rescale=1.1
        intermediate_index = ( ( F.relu(self.forward(dataset)) != 0).float() * (self.forward(dataset) != self.bias).float() >0).squeeze()
        if intermediate_index.float().sum() >0 :
            activation_pattern_of_intermediate_data = self.activation_pattern(dataset[intermediate_index])
            if activation_pattern_of_intermediate_data.max().item() >0 : ## max
                new_W0 = self.W(layer=0).detach()
                new_b0 = self.b(layer=0).detach()
                new_W1 = self.W(layer=1).detach()
                true_or_false = activation_pattern_of_intermediate_data.sum(dim=0)>0
                for i, boolean in enumerate(true_or_false):
                    if boolean:
                        new_W0[i] = new_W0[i] * rescale
                        new_b0[i] = new_b0[i] * rescale
                        new_W1[0][i] = new_W1[0][i] * rescale
                self.change_layer_weights(layer=0, W=new_W0, b=new_b0)
                self.change_layer_weights(layer=1, W=new_W1, b=None)
                text = "(rescaling)"
                print(f"\t{text:<15}In the polytope, {true_or_false.sum().item()} neurons are rescaled by {rescale}") if pt_option else None
        
        self.optimizer = optim.Adam(self.parameters(), lr=lr) # net. paramgeter 를 optimizer 가능하도록 넣어줘야해...!!

In [3]:
# Optimization setting
lr = 0.0001
criterion = nn.BCEWithLogitsLoss()

# finding the polytope cover !
repetition = 0
trn_X_index_all = (torch.ones(len(trn_X))>0).to(device) # Boolean tensor, shape = len(dataset)
trn_X_index = trn_X_index_all.clone()
# folder_name_above = output_path + f'/runs/{dataset}/' + datetime.datetime.now().strftime("%B%d_%H_%M_%S")
feature = "c_" if first_positive_init else "_"
folder_name_above = output_path + f'/runs/{dataset}/' + f"{selected_class}{feature}"
create_directory(folder_name_above) # create output folder
network_list =[]

complement = "^c" if first_positive_init else ""
print(f"We are finding polytope-basis cover of class [{selected_class}]"+complement+f" of {dataset}, n={n}, with random ratio={random_ratio}, with acc {Goal_Accuracy}")
positive_init = first_positive_init
# while trn_X_index[(trn_Y==0).view(-1)].sum()>0 and trn_X_index[(trn_Y==1).view(-1)].sum()>0 : # 100% accuracy
width_fail = list()
width_success = list()
width_difference = 1
# width_increment = int(input("### Please enter the width_increment: ###")) ## manually adjust the width

for _count in range(15):
    # set width
    if len(width_success) == 0 :
        width += width_increment # increment
    elif len(width_fail) == 0 :
        width = int (np.array(width_success).min()/2)
    else: 
        width = int( (np.array(width_fail).max() + np.array(width_success).min())/2 + torch.randint(min(width_difference , 10)+3, (1,)).item()/2 ) # intermediate value
#     width = int(input("### Please enter the network width: ###")) ################ manually adjust the width
    
    print("width_fail:", width_fail, "  ||    width_success:", width_success)
    print(f"Now width = {width}")
    net = polytope(width=width, positive_init=positive_init, small_norm_init=False).to(device)
    folder_name = folder_name_above + f"/Cover_{repetition+1}"
    writer = SummaryWriter(folder_name)
    
    # training
    trn_X_index_this_time = trn_X_index.clone()
    ####
#     trn_X_index_this_time[(trn_Y==float(positive_init)).view(-1)] = True  #### consider all data points in the other class
    ####
    net.train(repetition=repetition, trn_X_index= trn_X_index_this_time)
#     again = input("### Do you wanna train again? (y/n) ###")
#     if again == 'y':
#         net.fail = True
    if net.fail:
        # there is some problem for finding a cover in this training time.
        width_fail.append(net.width)
    else : # success !
            # otherwise, it was successful.
        # result
        print("Completed to find a polytope-basis cover.")
        print(net)
        width_success.append(net.width)
        network_list = []
        network_list.append(net) # add to the network_list
    repetition += 1
    
    if (len(width_fail) != 0) and (len(width_success) != 0) :
        width_difference = (np.array(width_success).min() - np .array(width_fail).max())
        if width_difference <= 1 :
            print("Found a single polytope cover")
            print("width_fail :", width_fail)
            print("width_success :", width_success)
            break
print("\n\n\n===== Found covers =====")
print("width_fail :", width_fail)
print("width_success :", width_success)

We are finding polytope-basis cover of class [1] of MNIST, n=60000, with random ratio=0.0, with acc 99.99
width_fail: []   ||    width_success: []
Now width = 40
=================================   Start Training. Repetition: 1   ==================================


 12%|█████████▏                                                                | 12484/100000 [01:50<12:51, 113.40it/s]

KeyboardInterrupt



In [ ]:
### Result ####
width_list = [net.width for net in network_list]
print(f"Output: a polytope-basis cover of class [{selected_class}]{complement} with random ratio {random_ratio}")
print(f"There are totally {len(network_list)} polytopes. The widths are",width_list,end="\n\n")

#### Accuracy #########
# trn acc
previous_trn_answer = torch.zeros_like(trn_Y)
for j, net in enumerate(network_list):
    previous_trn_answer += (-1)**j * (net(trn_X) == net.bias).float()
correct_trn_data = ((previous_trn_answer>0).float() == float(first_positive_init) + (1-2*(float(first_positive_init)))*trn_Y).sum().item()
trn_acc =  correct_trn_data / len(trn_Y) * 100
print(f"Trn ACC : {trn_acc:.2f} %  ||  # incorrect data : {len(trn_Y) - correct_trn_data} / {len(trn_Y)}")

# test ACC
previous_test_answer = torch.zeros_like(test_Y)
for j, net in enumerate(network_list):
    previous_test_answer += (-1)**j * (net(test_X) == net.bias).float()
correct_test_data = ((previous_test_answer>0).float() == float(first_positive_init) + (1-2*(float(first_positive_init)))*test_Y).sum().item()
test_acc =  correct_test_data / len(test_Y) * 100
print(f"Test ACC : {test_acc:.2f} %  ||  # incorrect data : {len(test_Y) - correct_test_data} / {len(test_Y)}")

f = open(f"./{folder_name_above}/Total_result.txt", 'w')
f.write(
f"""
    This is the result file
    
    # dataset
    dataset = {dataset}, {dataset_type}
    n = {len(trn_X)} # number of trained data
    d = {d} # dimension
    Initialization width = {width}
    data_balance = {data_balance}
    
    # class, convexity
    selected_class = {selected_class} # the class, used in this binary classification
    positive_init = {first_positive_init}
    Therefore, it finds a polytope-basis cover of class [{selected_class}]{complement})
    random ratio = {random_ratio}
    class_overlapped = {class_overlapped}
    
    width_fail : {width_fail}
    width_success : {width_success}
    
    # Accuracy
    Trn ACC : {trn_acc:.2f} %  ||  # incorrect data : {len(trn_Y) - correct_trn_data} / {len(trn_Y)}
    Test ACC : {test_acc:.2f} %  ||  # incorrect data : {len(test_Y) - correct_test_data} / {len(test_Y)}
    
    # optimization
    Epochs = {Epochs}
    Start_pruning = {start_pruning}
    
    There are totally {len(network_list)} polytopes. The widths are
    {width_list}
    
    The end.
"""
)
f.close()

print("\nwidth_fail :", width_fail)
print("width_success :", width_success)
## rename the folder name.
if len(width_success)>0:
    os.rename(folder_name_above, folder_name_above + str(np.array(width_success).min()))
else:
    os.rename(folder_name_above, folder_name_above + "inf")

In [ ]:
# ### For visualization ######
# for x,y in zip(trn_X, trn_Y):
# #     if net(x) == net.bias and y != 1:
#     if net(x) == net.bias and y != 1:
# #     if y == 1:
#         imshow(x.cpu().view(28,28), title=y.item())